In [24]:
import sys
sys.path.append("../../") 

In [25]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import max_error, mean_squared_error, r2_score
from modelos import *

Preparado de datos

In [26]:
paramsRandom = {
    "random_state": [RANDOM_SEED],
    'n_estimators': np.arange(1,602,100),
    'max_depth': [None, 4, 8, 12, 16, 20, 24, 28],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False]
}
paramsGrid = {
    "random_state": [RANDOM_SEED],
    'n_estimators': [100, 300, 600],
    'max_depth': [None, 16, 28],
    'min_samples_split': [2, 6],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

In [27]:
# Para el modelo con todas las variables
X_train, X_test, y_train, y_test = sep_train_test()
# Eliminamos las columnas repetidas y las columnas temporales (visto en el notebook del analisis)
X_train = X_train.drop(columns=["Temperatura", 'wspd', 'anio', 'mes', 'dia', 'hora'])
X_test = X_test.drop(columns=["Temperatura", 'wspd', 'anio', 'mes', 'dia', 'hora'])

Modelos

In [28]:
# Para probar con datos escalados
scaler = ColumnTransformer(
    transformers=[
        # no escalonamos las dummy de dirección de viento
        ("num", StandardScaler(), [c for c in X_train.columns if not c.startswith('wdir_')]),
    ],
    remainder='passthrough'
)
scaler.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['moonphase', 'temp', 'dewPt', 'heat_index',
                                  'rh', 'pressure', 'vis', 'wc', 'feels_like',
                                  'uv_index', 'Vviento', 'PeriodoOlas',
                                  'Lluvia', 'Nubosidad'])])

In [19]:
def random_forest(X_train, X_test, y_train, y_test, run_name, exp_info):
    flo = MLFlow("random_forest")

    modelo_basico = RandomForestRegressor(max_depth=10, random_state=RANDOM_SEED)
    cv_metrics = cross_validate(modelo_basico, X_train, y_train, cv=cv_folds(), scoring="neg_root_mean_squared_error", return_train_score=True)
    modelo_basico.fit(X_train, y_train)

    #Predicciones
    pred_modelo_basico = modelo_basico.predict(X_test)
    #print(f"Error cuadratico medio: {mean_squared_error(y_test, pred_modelo_basico)}.\nr2: {r2_score(y_test, pred_modelo_basico)}\nMaximo error {max_error(y_test, pred_modelo_basico)}")

    # metricas TEST
    metricas = calcular_metricas(y_test, pred_modelo_basico)

    # metricas CV
    metricas["CV_TEST_RMSE"] = -1 * cv_metrics['test_score'].mean()
    metricas["CV_TRAIN_RMSE"] = -1 * cv_metrics['train_score'].mean()
    print(metricas)

    flo.persist_model_to_mlflow(X_train, modelo_basico, {"random_state": RANDOM_SEED, "max_depth": 10}, metricas, run_name, exp_info)


In [20]:
def random_search(X_train, X_test, y_train, y_test, run_name, exp_info):
    flo = MLFlow("random_forest")
    modelo_random_search = RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=paramsRandom,n_iter=10, scoring='neg_mean_squared_error', cv=cv_folds(), random_state=RANDOM_SEED, return_train_score=True, n_jobs=-1)
    modelo_random_search.fit(X_train, y_train)
    mejor_modelo_randomSearch = modelo_random_search.best_estimator_

    #Predicciones
    #pred_modelo_random_search = mejor_modelo_randomSearch.predict(X_test)
    #print(f"Error cuadratico medio: {mean_squared_error(y_test, pred_modelo_random_search)}.\nr2: {r2_score(y_test, pred_modelo_random_search)}\nMaximo error {max_error(y_test, pred_modelo_random_search)}")

    metricas = calcular_metricas_search(modelo_random_search, X_test, y_test)
    print(metricas)

    flo.persist_model_to_mlflow(X_train, mejor_modelo_randomSearch, modelo_random_search.best_params_, metricas, run_name, exp_info)

In [21]:
def grid_search(X_train, X_test, y_train, y_test, run_name, exp_info):
    flo = MLFlow("random_forest")
    modelo_grid_search = GridSearchCV(estimator=RandomForestRegressor(),param_grid=paramsGrid, scoring='neg_mean_squared_error', cv=cv_folds(), return_train_score=True, n_jobs=-1)
    modelo_grid_search.fit(X_train, y_train)
    mejor_modelo_gridSearch = modelo_grid_search.best_estimator_

    #Predicciones
    #pred_modelo_grid_search = mejor_modelo_gridSearch.predict(X_test)
    #print(f"Error cuadratico medio: {mean_squared_error(y_test, pred_modelo_grid_search)}.\nr2: {r2_score(y_test, pred_modelo_grid_search)}\nMaximo error {max_error(y_test, pred_modelo_grid_search)}")
    
    metricas = calcular_metricas_search(modelo_grid_search, X_test, y_test)
    print(metricas)
    
    flo.persist_model_to_mlflow(X_train, mejor_modelo_gridSearch, modelo_grid_search.best_params_, metricas, run_name, exp_info)



RANDOM FOREST BÁSICO

In [12]:
#Con las variables explicativas sin escalar
random_forest(X_train, X_test, y_train, y_test, run_name="random_forest_basico", exp_info="Random Forest con todos los hiperparámetros predeterminados y sin escalado de las variables explicativas") 

2024/04/28 21:12:41 INFO mlflow.tracking.fluent: Experiment with name 'random_forest' does not exist. Creating a new experiment.


{'TEST_MAX_ERROR': 2.407003176282278, 'TEST_ROOT_MEAN_SQ_ERROR': 0.38695458547829115, 'TEST_MEDIAN_ABS_ERROR': 0.18807589004350211, 'TEST_MEAN_ABS_ERROR': 0.27240766990630233, 'CV_TEST_RMSE': 0.2627375222484812, 'CV_TRAIN_RMSE': 0.1942708758271285}


/Users/tyomikjan/UNIVERSITY/PD1/PD/venv/lib/python3.11/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'random_forest_basico'.
2024/04/28 21:13:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_basico

In [13]:
#Con las variables explicativas escaladas
random_forest(scaler.transform(X_train), scaler.transform(X_test), y_train, y_test, run_name="esc_random_forest_basico", exp_info="Random Forest con todos los hiperparámetros predeterminados y con escalado de las variables explicativas") 

KeyboardInterrupt: 

RANDOM FOREST CON RANDOM SEARCH

In [17]:
#Con las variables explicativas sin escalar
random_search(X_train, X_test, y_train, y_test, run_name="rf_con_random_search", exp_info="Random Forest donde los hiperparámetros se escogen mediante random search y sin escalado de las variables explicativas")

{'TEST_MAX_ERROR': 1.9861018162324129, 'TEST_ROOT_MEAN_SQ_ERROR': 0.4109153323030973, 'TEST_MEDIAN_ABS_ERROR': 0.19879180624516402, 'TEST_MEAN_ABS_ERROR': 0.2899906532685502, 'CV_TEST_RMSE': 0.06707166347149676, 'CV_TRAIN_RMSE': 0.027438585316847335}


/Users/tyomikjan/UNIVERSITY/PD1/PD/venv/lib/python3.11/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/tyomikjan/UNIVERSITY/PD1/PD/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module i

In [18]:
#Con las variables explicativas escaladas
random_search(scaler.transform(X_train), scaler.transform(X_test), y_train, y_test, run_name="esc_rf_con_random_search", exp_info="Random Forest donde los hiperparámetros se escogen mediante random search y con escalado de las variables explicativas")

{'TEST_MAX_ERROR': 1.9785163130125807, 'TEST_ROOT_MEAN_SQ_ERROR': 0.4094530505161749, 'TEST_MEDIAN_ABS_ERROR': 0.19786944823215677, 'TEST_MEAN_ABS_ERROR': 0.28856151227097776, 'CV_TEST_RMSE': 0.0671882512990779, 'CV_TRAIN_RMSE': 0.027452590512611973}


/Users/tyomikjan/UNIVERSITY/PD1/PD/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/tyomikjan/UNIVERSITY/PD1/PD/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'rf_con_random_search' already exists. Creating a new version of this model...
2024/04/16 18:29:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rf_con_random_search, version 2
Creat

RANDOM FOREST CON GRID SEARCH

In [ ]:
#Con las variables explicativas sin escalar
grid_search(X_train, X_test, y_train, y_test, run_name="rf_con_grid_search", exp_info="Random Forest donde los hiperparámetros se escogen mediante gridsearch y sin escalado de las variables explicativas")

/Users/tyomikjan/UNIVERSITY/PD1/PD/venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
#Con las variables explicativas escaladas
grid_search(scaler.transform(X_train), scaler.transform(X_test), y_train, y_test, run_name="esc_rf_con_grid_search", exp_info="Random Forest donde los hiperparámetros se escogen mediante gridsearch y con escalado de las variables explicativas")